In [4]:
import MDAnalysis as mda
from MDAnalysis.analysis import helix_analysis as hel
import pandas as pd
import nglview as nv
from IPython.display import display
import warnings
import os

%matplotlib inline

In [5]:
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")


In [11]:
# Process all files
# pdb_files = [
#     "aligned_holo_out_open_8jts.pdb",
#     "aligned_holo_out_occ_8jtt.pdb",
#     "aligned_holo_in_occ_8jtv.pdb",
#     "aligned_holo_in_open_8sc4.pdb"
# ]
pdb_files = ["aligned_apo_out_open_8et6.pdb", 
             "aligned_holo_out_open_8jts.pdb", 
             "aligned_holo_out_occ_8jtt.pdb", 
             "aligned_holo_in_occ_8jtv.pdb", 
             "aligned_holo_in_open_8sc4.pdb", 
             "aligned_apo_in_open_8sc1.pdb"]


In [12]:
# Define TM helix ranges (residue numbers)
tm_ranges = [
    (15, 42),    # TM1
    (146, 170),  # TM2
    (177, 195),  # TM3
    (200, 228),  # TM4
    (233, 257),  # TM5
    (261, 283),  # TM6
    (341, 368),  # TM7
    (377, 397),  # TM8
    (403, 423),  # TM9
    (431, 455),  # TM10
    (462, 490),  # TM11
    (493, 513)   # TM12
]


In [13]:
def analyze_helix_tilts(inp_pdb, tm_ranges):
    """Analyze tilt angles for all transmembrane helices"""
    u = mda.Universe(inp_pdb)
    results = {'system': os.path.basename(inp_pdb)[8:-4]}
    
    for idx, (start, end) in enumerate(tm_ranges, 1):
        try:
            h = hel.HELANAL(
                u,
                select=f'name CA and resnum {start}-{end}',
                ref_axis=[0, 0, 1]
            ).run()
            
            # Store rounded results
            tilt_mean = round(h.results.summary['global_tilts']['mean'], 2)
            results[f'tm{idx}_tilt'] = tilt_mean
            
        except Exception as e:
            print(f"Error processing TM{idx} ({start}-{end}): {str(e)}")
            results[f'tm{idx}_tilt'] = None
    
    return results


In [15]:
# Create DataFrame
df = pd.DataFrame([analyze_helix_tilts(f, tm_ranges) for f in pdb_files])

# # Save to CSV
df.to_csv('oct1_tm_helix_tilts.csv', index=False)

# Display results
print("\nFinal DataFrame:")
display(df)


Final DataFrame:


,system,tm1_tilt,tm2_tilt,tm3_tilt,tm4_tilt,tm5_tilt,tm6_tilt,tm7_tilt,tm8_tilt,tm9_tilt,tm10_tilt,tm11_tilt,tm12_tilt
0,apo_out_open_8et6,86.03,88.54,112.14,68.18,120.61,87.78,98.65,63.58,78.90,103.50,67.66,88.44
1,holo_out_open_8jts,82.56,93.20,106.60,69.92,120.19,90.72,101.95,63.86,74.96,107.07,70.85,85.88
2,holo_out_occ_8jtt,81.77,92.66,106.88,69.96,119.23,91.62,104.55,66.86,81.29,104.27,75.69,83.23
3,holo_in_occ_8jtv,71.67,100.87,97.01,76.26,113.92,104.90,116.85,63.18,93.82,94.01,85.09,69.80
4,holo_in_open_8sc4,75.46,96.60,96.61,75.35,115.37,107.57,117.08,62.95,92.66,92.21,86.79,70.34
5,apo_in_open_8sc1,74.67,97.93,97.26,74.95,114.88,104.42,116.52,63.07,93.49,92.09,86.11,69.93
